In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
input_df = pd.read_csv('/kaggle/input/title-conference/title_conference.csv')

In [ ]:
input_df.columns

In [ ]:
input_df.head(4)

In [ ]:
selected_df = input_df.copy()

In [ ]:
num_cat = []
un_cat = set(selected_df['Conference'].values.tolist())
i = 0
for z in un_cat:
    num_cat.append(i)
    i = i+1

In [ ]:
cat_dict = dict(zip(un_cat,num_cat))

In [ ]:
def cat(x):
    return cat_dict[x]

selected_df['label'] = selected_df['Conference'].apply(cat)

In [ ]:
selected_df['Title'][0]

In [ ]:
!pip install keras-bert

In [ ]:
import pandas as pd
import numpy as np
from keras.utils import np_utils
import tensorflow as tf
import keras as keras
import keras.backend as K
from keras.models import load_model
from keras.layers.merge import concatenate
from keras_bert import load_trained_model_from_checkpoint, load_vocabulary
from keras_bert import Tokenizer
from keras_bert import AdamWarmup, calc_train_steps
from keras.layers import Input
import pandas as pd
import numpy as np
from tqdm import tqdm
import gc
import transformers
from kaggle_datasets import KaggleDatasets
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

In [ ]:
selected_df.head()

In [ ]:
SEQ_LEN = 128
BATCH_SIZE = 32
EPOCHS = 10
LR = 1e-4
import os
pretrained_path = '/kaggle/input/bert-base-multilingual/multilingual_L-12_H-768_A-12/'
config_path = os.path.join(pretrained_path, 'bert_config.json')
checkpoint_path = os.path.join(pretrained_path, 'bert_model.ckpt')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')

In [ ]:
token_dict = load_vocabulary(vocab_path)
tokenizer = Tokenizer(token_dict)

In [ ]:
def convert_data(test_df,DATA_COLUMN):
    global tokenizer
    indices = []
    for i in tqdm(range(len(test_df))):
        ids, segments = tokenizer.encode(test_df[DATA_COLUMN].iloc[i], max_len=SEQ_LEN)
        indices.append(ids)
    indices = np.array(indices)
    return [indices, np.zeros_like(indices)]

In [ ]:
def build_model():
    from keras.layers.normalization import BatchNormalization
    model = load_trained_model_from_checkpoint(
        config_path,
        checkpoint_path,
        training=True,
        trainable=True,
        seq_len=SEQ_LEN,
    )

    inputs = model.inputs[:2]
    dense = model.layers[-3].output
    dense2 = keras.layers.Dense(10,activation='relu', kernel_initializer ='glorot_uniform')(dense)
    dense3 = keras.layers.Dense(10,activation='relu', kernel_initializer ='glorot_uniform')(dense2)
    outputs = keras.layers.Dense(1, activation='sigmoid', kernel_initializer='glorot_uniform',
                                 name = 'real_output')(dense3)

    

    model = keras.models.Model(inputs, outputs)
       
    return model

In [ ]:
def cleansing(x):
    quoteRemoval = x.replace('"','')
    spaceRemoval = re.sub("\s\s+" , " ", quoteRemoval)
    stringRemoval = spaceRemoval.strip()
    urlRemove = re.sub(r'http\S+', '', stringRemoval)
    specialChar = re.sub(r"[^a-zA-Z]+", ' ',urlRemove) 
    return specialChar

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


selected_df['Title'] = selected_df['Title'].apply(cleansing)


In [ ]:
X = selected_df['Title'].values
Y = selected_df['label'].values

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)


In [ ]:
df_train = pd.DataFrame(X_train, columns = ['Title'])
df_test = pd.DataFrame(X_test, columns = ['Title'])

In [ ]:
X = convert_data(df_train,'Title')
X_test = convert_data(df_test,'Title')


In [ ]:
y_train.shape[0]

In [ ]:
model = build_model()
model.summary()
decay_steps, warmup_steps = calc_train_steps(y_train.shape[0],batch_size=BATCH_SIZE,epochs=EPOCHS,)
model.compile(AdamWarmup(decay_steps=decay_steps, warmup_steps=warmup_steps, lr=LR),loss='categorical_crossentropy',metrics=['acc',f1_m,precision_m, recall_m])

In [ ]:
from keras.callbacks import *
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=2, min_lr=1e-7, verbose=1)
BERT = model.fit(
        X,
        y_train,
        epochs=10,
        batch_size=BATCH_SIZE,
        validation_split=0.2,
        callbacks=[reduce_lr]
    )